In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from utils.convex_sets import Singleton, Polyhedron, Ellipsoid
from utils.convex_functions import TwoNorm, SquaredTwoNorm
from utils.graph import GraphOfConvexSets
from utils.shortest_path import ShortestPathProblem

In [ ]:
# convex sets
singletons = (
    Singleton((0, 0)),
    Singleton((9, 0)),
)
polyhedra = (
    Polyhedron.from_vertices(([1, 0], [1, -2], [3, -2], [3, -1])),
    Polyhedron.from_vertices(([4, -2], [5, -4], [3, -4], [2, -3])),
    Polyhedron.from_vertices(([2, 2], [1, 3], [2, 4], [4, 4], [4, 3])),
    Polyhedron.from_vertices(([5, 4], [7, 4], [6, 3])),
    Polyhedron.from_vertices(([7, 2], [8, 2], [9, 3], [8, 4])),
)
ellipsoids = (
    Ellipsoid((4, 1), ([1, 0], [0, 1])),
    Ellipsoid((7, -2), ([.25, 0], [0, 1])),
)
sets = singletons + polyhedra + ellipsoids

# label for the vertices
vertices = ['s', 't']
vertices += [f'p{i}' for i in range(len(polyhedra))]
vertices += [f'e{i}' for i in range(len(ellipsoids))]

# add convex sets to the graph
G = GraphOfConvexSets()
G.add_sets(sets, vertices)
G.set_source('s')
G.set_target('t')

# edges
H = np.hstack((np.eye(2), -np.eye(2)))
l = TwoNorm(H)
edges = {
    's': ('p0', 'p1', 'p2'),
    'p0': ('e1',),
    'p1': ('p2', 'e0', 'e1'),
    'p2': ('p1', 'p3', 'e0'),
    'p3': ('t', 'p2', 'p4', 'e1'),
    'p4': ('t', 'e0'),
    'e0': ('p3', 'p4', 'e1'),
    'e1': ('t', 'p4', 'e0')
}
for u, vs in edges.items():
    for v in vs:
        G.add_edge(u, v, l)
        
# draw convex sets and edges
plt.figure()
G.draw_sets()
G.draw_edges()
G.label_sets()

In [ ]:
G.graphviz()

In [ ]:
spp = ShortestPathProblem(G, relaxation=0)
sol = spp.solve()

print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(sol.primal.phi[k]), 4)
    print(edge, flow)

In [ ]:
# edge lenghts
l2 = SquaredTwoNorm(H)
G2 = deepcopy(G)
for e in G2.edges:
    G2.set_edge_length(e, l2)

spp2 = ShortestPathProblem(G2, relaxation=0)
sol2 = spp2.solve()

print('Cost:', sol2.cost)
print('\nFlows:')
for k, edge in enumerate(G2.edges):
    flow = round(abs(sol2.primal.phi[k]), 4)
    print(edge, flow)

In [ ]:
plt.figure()
G.draw_sets()
G.draw_edges()

offset = np.array([0, -.25])
plt.text(*(G.source_set.center + offset), r'$X_s$', ha='center', va='top')
plt.text(*(G.target_set.center + offset), r'$X_t$', ha='center', va='top')

plt.plot([np.nan] * 2, 'b--', label='Euclidean distance')
plt.plot([np.nan] * 2, 'r-.', label='Euclidean distance squared')
G.draw_path(sol.primal.phi, sol.primal.x, color='b', linestyle='--')
G.draw_path(sol2.primal.phi, sol2.primal.x, color='r', linestyle='-.')

plt.xticks(range(10))
plt.legend(loc='lower center', bbox_to_anchor=(0.5, 1.0))
plt.grid()

In [ ]:
scales = np.logspace(-2, 2, 200)
c_micp = []
c_relaxation = []
c_micp2 = []
c_relaxation2 = []
for s in scales:
    
    G_scaled = deepcopy(G)
    G_scaled.scale(s)
    spp = ShortestPathProblem(G_scaled, relaxation=0)
    c_micp.append(spp.solve().cost)
    spp = ShortestPathProblem(G_scaled, relaxation=1)
    c_relaxation.append(spp.solve().cost)
    
    G2_scaled = deepcopy(G2)
    G2_scaled.scale(s)
    spp = ShortestPathProblem(G2_scaled, relaxation=0)
    c_micp2.append(spp.solve().cost)
    spp = ShortestPathProblem(G2_scaled, relaxation=1)
    c_relaxation2.append(spp.solve().cost)

In [ ]:
def micp_vs_relaxation(c_micp, c_relaxation):
    plt.plot(scales, c_micp, label='MICP', linestyle='-', linewidth=2)
    plt.plot(scales, c_relaxation, label='Convex relaxation', linestyle='--', linewidth=2)
    plt.xlabel(r'Scale factor $r$')
    plt.ylabel('Cost')
    plt.xlim([scales[0], scales[-1]])
    plt.xscale('log')
    plt.grid(1)
    plt.legend()

In [ ]:
plt.figure(figsize=(5, 3))
micp_vs_relaxation(c_micp, c_relaxation)

In [ ]:
plt.figure(figsize=(5, 3))
micp_vs_relaxation(c_micp2, c_relaxation2)